In [20]:
import pandas as pd
from langdetect import detect

def detect_en_lang(title, description):
    try:
        return detect(title) == 'en' and detect(description) == 'en'
    except:
        return False

    
# cavideos = pd.read_csv('dataset/CAvideos.csv')
# devideos = pd.read_csv('dataset/DEvideos.csv')
# frvideos = pd.read_csv('dataset/FRvideos.csv')
# gbvideos = pd.read_csv('dataset/GBvideos.csv')
# usvideos = pd.read_csv('dataset/USvideos.csv')
dataset = pd.read_csv('dataset/en_language_dataset.csv')
# dataset = dataset.append(cavideos).append(devideos).append(frvideos).append(gbvideos).append(usvideos)
# dataset = dataset[dataset.apply(lambda x: detect_en_lang(str(x['title']), str(x['description'])), axis=1)]
dataset.count()

Unnamed: 0                74800
video_id                  74065
trending_date             74065
title                     74065
channel_title             74065
category_id               74065
publish_time              74065
tags                      74065
views                     74065
likes                     74065
dislikes                  74065
comment_count             74065
thumbnail_link            74065
comments_disabled         74065
ratings_disabled          74065
video_error_or_removed    74065
description               74065
dtype: int64

In [21]:
import datetime

trending_date = pd.to_datetime(dataset['trending_date'], format='%y.%d.%m')
publish_time = pd.to_datetime(dataset['publish_time'])
trending_publish_diff = (trending_date - publish_time).map(lambda x: x.seconds / 3600)
dataset['trending_publish_diff'] = trending_publish_diff

dataset.head()

,Unnamed: 0,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,trending_publish_diff
0,0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10.0,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579.0,787425.0,43420.0,125882.0,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,6.999167
1,2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23.0,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434.0,146035.0,5339.0,8181.0,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,4.910000
2,4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10.0,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622.0,1634130.0,21082.0,85067.0,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,12.929444
3,5,0yIWz1XEeyc,17.14.11,Jake Paul Says Alissa Violet CHEATED with LOGA...,DramaAlert,25.0,2017-11-13T07:37:51.000Z,"#DramaAlert|""Drama""|""Alert""|""DramaAlert""|""keem...",1309699.0,103755.0,4613.0,12143.0,https://i.ytimg.com/vi/0yIWz1XEeyc/default.jpg,False,False,False,► Follow for News! - https://twitter.com/KEEMS...,16.369167
4,6,_uM5kFfkhB8,17.14.11,Vanoss Superhero School - New Students,VanossGaming,23.0,2017-11-12T23:52:13.000Z,"Funny Moments|""Montage video games""|""gaming""|""...",2987945.0,187464.0,9850.0,26629.0,https://i.ytimg.com/vi/_uM5kFfkhB8/default.jpg,False,False,False,Vanoss Merch Shop: https://vanoss.3blackdot.co...,0.129722


In [22]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit_transform(dataset['channel_title'].fillna(''))
dataset['channel_title'] = le.fit_transform(dataset['channel_title'].fillna(''))
dataset.head()

,Unnamed: 0,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,trending_publish_diff
0,0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,1197,10.0,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579.0,787425.0,43420.0,125882.0,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,6.999167
1,2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",3333,23.0,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434.0,146035.0,5339.0,8181.0,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,4.910000
2,4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),1165,10.0,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622.0,1634130.0,21082.0,85067.0,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,12.929444
3,5,0yIWz1XEeyc,17.14.11,Jake Paul Says Alissa Violet CHEATED with LOGA...,1099,25.0,2017-11-13T07:37:51.000Z,"#DramaAlert|""Drama""|""Alert""|""DramaAlert""|""keem...",1309699.0,103755.0,4613.0,12143.0,https://i.ytimg.com/vi/0yIWz1XEeyc/default.jpg,False,False,False,► Follow for News! - https://twitter.com/KEEMS...,16.369167
4,6,_uM5kFfkhB8,17.14.11,Vanoss Superhero School - New Students,4320,23.0,2017-11-12T23:52:13.000Z,"Funny Moments|""Montage video games""|""gaming""|""...",2987945.0,187464.0,9850.0,26629.0,https://i.ytimg.com/vi/_uM5kFfkhB8/default.jpg,False,False,False,Vanoss Merch Shop: https://vanoss.3blackdot.co...,0.129722


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score

X = dataset[['channel_title', 'category_id']]
y = dataset[['views', 'likes', 'dislikes']]
X = X.dropna(how='any')
y = y.dropna(how='any')
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, test_size=0.5, random_state=0)
X_test, X_development, y_test, y_development = train_test_split(X_test, y_test, train_size=0.5, test_size=0.5, random_state=0)

knn = KNeighborsRegressor(n_neighbors=15)
regr = MultiOutputRegressor(knn)
regr.fit(X_train, y_train)
y_test_pred = regr.predict(X_test)
y_development_pred = regr.predict(X_development)
r2_score(y_development, y_development_pred, multioutput='raw_values')

array([0.37100375, 0.58813864, 0.66437409])

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(dataset.title.fillna(''))
idf = vectorizer.idf_
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

video_id                  29340
trending_date             29340
title                     29340
channel_title             29340
category_id               29340
publish_time              29340
tags                      29340
views                     29340
likes                     29340
dislikes                  29340
comment_count             29340
thumbnail_link            29340
comments_disabled         29340
ratings_disabled          29340
video_error_or_removed    29340
description               28219
dtype: int64